## general import and functions

In [2]:
from pathlib import Path
import pickle as pkl
from collections import defaultdict, Counter
from itertools import permutations, combinations
from functools import reduce
import numpy as np
import os

In [1]:
def pkl_save(data, file):
    with open(file, "wb") as f:
        pkl.dump(data, f)
        
def pkl_load(file):
    with open(file, "rb") as f:
        data = pkl.load(f)
    return data

def load_text(ifn):
    with open(ifn, "r") as f:
        txt = f.read()
    return txt

In [3]:
import sys
sys.path.append("/home/zehao.yu/workspace/py3/SDoH/NLPpreprocessing-master/")

In [4]:
from annotation2BIO import pre_processing, read_annotation_brat, generate_BIO

In [5]:
MIMICIII_PATTERN = "\[\*\*|\*\*\]"

In [6]:
def parse_brat_ner(brat_data):
	# assert re.match("^T[0-9]+\t[A-Za-z]+ [0-9]+ [0-9]+\t\.+$", brat_data), "invalid brat format for {}".format(brat_data)
	info = brat_data.split("\t")
	idx = info[0]
	text = info[2]
	tse = info[1]
	if ";" in tse:
		ii = tse.split(" ")
		return [idx, ii[0], " ".join(ii[1:-1]), ii[-1], text]
	else:
		tag, s, e = tse.split(" ")
		return [idx, tag, int(s), int(e), text]


def parse_brat_rel(brat_data):
	# assert re.match("^R[0-9]+\t[A-Za-z]+-[A-Za-z]+ Arg1:T[0-9]+ Arg2:T[0-9]+$", brat_data), "invalid brat format for {}".format(brat_data)
	info = brat_data.split("\t")
	idx = info[0]
	tag, arg1, arg2 = info[1].split(" ")
	arg1 = arg1.split(":")[-1]
	arg2 = arg2.split(":")[-1]
	return [idx, tag, arg1, arg2]


def read_brat(file_name):
	ners = []
	rels = []

	with open(file_name, "r") as f:
		cont = f.read().strip()
	
	if not cont:
		return ners, rels

	# process ner and relation
	for each in cont.split("\n"):
		if each.startswith("T"):
			ners.append(parse_brat_ner(each))
		elif each.startswith("R"):
			rels.append(parse_brat_rel(each))
		# elif each.startswith("#"):
		# 	continue
		else:
			continue
			# raise RuntimeError('invalid brat data: {}'.format(each))
	return ners, rels

## relation generation functions

In [7]:
def get_all_rel_types(root, valid_comb):
    root = Path(root)
    rtt = []
    for ann_fn in root.glob("*.ann"):
#         _, rels = read_brat(ann_fn)
        _, _, rels = read_annotation_brat(ann_fn)
        #('Strength-Drug', 'T3', 'T2')
        for rel in rels:
#             rtype = rel[1]
            rtype = rel[0]
            if tuple(rtype.split("-")) not in valid_comb:
                continue
            rtt.append(rtype)
    rs, rc = set(rtt), Counter(rtt).most_common()
    r2i = {NEG_REL: '0'}
    for idx, rt in enumerate(rs):
        r2i[rt] = str(idx + 1)
    i2r = {v: k for k, v in r2i.items()}
    return r2i, i2r, rc

In [8]:
def create_entity_to_sent_mapping(nnsents, entities, idx2e):
    loc_ens = []
    
    ll = len(nnsents)
    mapping = defaultdict(list)
    for idx, each in enumerate(entities):
        en_label = idx2e[idx]
        en_s = each[2][0]
        en_e = each[2][1]
        new_en = []
        
        i = 0
        while i < ll and nnsents[i][1][0] < en_s:
            i += 1
        s_s = nnsents[i][1][0]
        s_e = nnsents[i][1][1]

        if en_s == s_s:
            mapping[en_label].append(i)

            while i < ll and s_e < en_e:
                i += 1
                s_e = nnsents[i][1][1]
            if s_e == en_e:
                 mapping[en_label].append(i)
            else:
                mapping[en_label].append(i)
                print("last index not match ", each)
        else:
            mapping[en_label].append(i)
            print("first index not match ", each)

            while i < ll and s_e < en_e:
                i += 1
                s_e = nnsents[i][1][1]
            if s_e == en_e:
                 mapping[en_label].append(i)
            else:
                mapping[en_label].append(i)
                print("last index not match ", each)
    return mapping

def get_permutated_relation_pairs(eid2idx):
    all_pairs = []
    all_ids = [k for k, v in eid2idx.items()]
    for e1, e2 in permutations(all_ids, 2):
        all_pairs.append((e1, e2))
    return all_pairs

In [9]:
"""
procedure:

1. preprocess text into sentences
2. find each entity associated sentence idx
3. generate entity pairs as relation candidates
4. extract eneity associated sentences, locate entities in pair and insert special tags
5. save generated data

result output:

1. keep pos predicted relations
2. using map files to locate relation associated entities
3. output as brat
"""
def validate_rels(rels, valid):
    nrels = []
    for rel in rels:
        rtype = rel[0]
        if tuple(rtype) not in valid:
            print("invalid: ", rel)
            continue
        nrels.append(rel)
    return nrels


def check_tags(s1, s2):
    assert EN1_START in s1 and EN1_END in s1, f"tag error: {s1}"
    assert EN2_START in s2 and EN2_END in s2, f"tag error: {s2}"


def format_relen(en, rloc, nsents):
    if rloc == 1:
        spec1, spec2 = EN1_START, EN1_END
    else:
        spec1, spec2 = EN2_START, EN2_END
    sn1, tn1 = en[0][3]
    sn2, tn2 = en[-1][3]
    target_sent = nsents[sn1]
    target_sent = [each[0] for each in target_sent]
    ors =  " ".join(target_sent)
    
    if sn1 != sn2:
#         print("[!!!Warning] The entity is not in the same sentence\n", en)
        tt = nsents[sn2]
        tt = [each[0] for each in tt]
        target_sent.insert(tn1, spec1)
        tt.insert(tn2+1, spec2)
        target_sent = target_sent + tt
#         print(target_sent)
    else:
        target_sent.insert(tn1, spec1)
        target_sent.insert(tn2+2, spec2)
    
    fs = " ".join(target_sent)
    
    return sn1, sn2, fs, ors


def gene_true_relations(rels, mappings, ens, e2i, nnsents, nsents, valid_comb, fid=None):
    true_pairs = set()
    pos_samples = []
    
    for rel in rels:
        rel_type = rel[0]
        enid1, enid2 = rel[1:]
        """
        [['100', (15443, 15446), (16473, 16476), (231, 4), 'B-Strength'], 
        ['mg', (15447, 15449), (16477, 16479), (231, 5), 'I-Strength']] 
        [['Metoprolol', (15422, 15432), (16452, 16462), (231, 2), 'B-Drug'], 
        ['Succinate', (15433, 15442), (16463, 16472), (231, 3), 'I-Drug']]
        """
#         enbs1, enbe1 = mappings[enid1]
#         en1 = nnsents[enbs1: enbe1+1]
#         si1, fs1, ors1 = format_relen(en1, 1, nsents)
#         enbs2, enbe2 = mappings[enid2]
#         en2 = nnsents[enbs2: enbe2+1]
#         si2, fs2, ors2 = format_relen(en2, 2, nsents)
#         sent_diff = abs(si1 - si2)
        
        
        enbs1, enbe1 = mappings[enid1]
        en1 = nnsents[enbs1: enbe1+1]
        si1, sii1, fs1, ors1 = format_relen(en1, 1, nsents)
        enbs2, enbe2 = mappings[enid2]
        en2 = nnsents[enbs2: enbe2+1]
        si2, sii2, fs2, ors2 = format_relen(en2, 2, nsents)
        sent_diff = abs(si1 - si2)
        
        en1t = en1[0][-1].split("-")[-1]
        en2t = en2[0][-1].split("-")[-1]
        
#         print(abs(si1 - si2), abs(sii1 - sii2), abs(sii1 - si2), abs(si1 - sii2))
        
        true_pairs.add((enid1, enid2))
        
        if (en1t, en2t) not in valid_comb:
            continue
        
        if sent_diff <= CUTOFF:
            check_tags(fs1, fs2)
            assert (en1t, en2t) in valid_comb, f"{en1t} {en2t}"
            if fid:
                pos_samples.append((sent_diff, rel_type, fs1, fs2, en1t, en2t, enid1, enid2, fid))
#                 pos_samples.append((sent_diff, "pos", fs1, fs2, en1t, en2t, enid1, enid2, fid))
            else:
                pos_samples.append((sent_diff, rel_type, fs1, fs2, en1t, en2t, enid1, enid2))
#                 pos_samples.append((sent_diff, "pos", fs1, fs2, en1t, en2t, enid1, enid2))
#         print(sent_diff, rel_type, fs1, fs2, ors1, ors2)
    
    return pos_samples, true_pairs
        

def gene_neg_relation(perm_pairs, true_pairs, mappings, ens, e2i, nnsents, nsents, valid_comb, fid=None):
    neg_samples = []
    for each in perm_pairs:
        enid1, enid2 = each
        
        # not in true relation
        if (enid1, enid2) in true_pairs:
            continue
        
        enc1 = ens[e2i[enid1]]
        enc2 = ens[e2i[enid2]]
    
        #('Metoprolol succinate', 'Drug', (14660, 14680))
        
        enbs1, enbe1 = mappings[enid1]
        en1 = nnsents[enbs1: enbe1+1]
        si1, sii1, fs1, ors1 = format_relen(en1, 1, nsents)
        enbs2, enbe2 = mappings[enid2]
        en2 = nnsents[enbs2: enbe2+1]
        si2, sii2, fs2, ors2 = format_relen(en2, 2, nsents)
        sent_diff = abs(si1 - si2)
        
        en1t = en1[0][-1].split("-")[-1]
        en2t = en2[0][-1].split("-")[-1]
        
#         print((enc1[1], enc2[1]), (en1t, en2t))
        
        if (en1t, en2t) not in valid_comb:
            continue
        
        if sent_diff <= CUTOFF:
            check_tags(fs1, fs2)
            assert (en1t, en2t) in valid_comb, f"{en1t} {en2t}"
            if fid:
                neg_samples.append((sent_diff, NEG_REL, fs1, fs2, en1t, en2t, enid1, enid2, fid))
            else:
                neg_samples.append((sent_diff, NEG_REL, fs1, fs2, en1t, en2t, enid1, enid2))
    
    return neg_samples

    
def create_training_samples(file_path, valids=None, valid_comb=None):
    fids = []
    root = Path(file_path)
    
    dpos = defaultdict(list)
    dneg = defaultdict(list)
    
    for txt_fn in root.glob("*.txt"):
        fids.append(txt_fn.stem)
        ann_fn = root / (txt_fn.stem+".ann")

        # load text
        txt = load_text(txt_fn)
        pre_txt, sents = pre_processing(txt_fn, deid_pattern=MIMICIII_PATTERN)
#         pre_txt = pre_txt.split("\n")
        e2i, ens, rels = read_annotation_brat(ann_fn)
#         ens, rels = read_brat(ann_fn)
        i2e = {v: k for k, v in e2i.items()}
        
#         rels = validate_rels(rels, n2c2_valid_comb)
        
        nsents, sent_bound = generate_BIO(sents, ens, file_id="", no_overlap=False, record_pos=True)
#         print(nsents)
        total_len = len(nsents)
        nnsents = [w for sent in nsents for w in sent]
        mappings = create_entity_to_sent_mapping(nnsents, ens, i2e)

        pos_samples, true_pairs = gene_true_relations(
            rels, mappings, ens, e2i, nnsents, nsents, valid_comb, fid=txt_fn.stem)
        perm_pairs = get_permutated_relation_pairs(e2i)
        neg_samples = gene_neg_relation(
            perm_pairs, true_pairs, mappings, ens, e2i, nnsents, nsents, valid_comb, fid=txt_fn.stem)
        
        for pos_sample in pos_samples:
            dpos[pos_sample[0]].append(pos_sample)
        for neg_sample in neg_samples:
            dneg[neg_sample[0]].append(neg_sample)
#         break
        
    return dpos, dneg


def create_test_samples(file_path, valids=None, valid_comb=None):
    #create a separate mapping file
    rel_mappings = []
    #
    fids = []
    root = Path(file_path)
    preds = defaultdict(list)
    
    g_idx = 0
    
    for txt_fn in root.glob("*.txt"):
        fids.append(txt_fn.stem)
        ann_fn = root / (txt_fn.stem + ".ann")

        # load text
        txt = load_text(txt_fn)
        pre_txt, sents = pre_processing(txt_fn, deid_pattern=MIMICIII_PATTERN)
        e2i, ens, _ = read_annotation_brat(ann_fn)
        i2e = {v: k for k, v in e2i.items()}
        
        nsents, sent_bound = generate_BIO(sents, ens, file_id="", no_overlap=False, record_pos=True)
        total_len = len(nsents)
        nnsents = [w for sent in nsents for w in sent]
        mappings = create_entity_to_sent_mapping(nnsents, ens, i2e)
        
        perm_pairs = get_permutated_relation_pairs(e2i)
        pred = gene_neg_relation(perm_pairs, set(), mappings, ens, e2i, nnsents, nsents, valid_comb, fid=txt_fn.stem)
        for idx, pred_s in enumerate(pred):
            preds[pred_s[0]].append(pred_s)
#             rel_mappings.append((g_idx, *pred_s[-3:]))
#             g_idx += 1
            
    return preds #rel_mappings

In [32]:
"""
Output strategy:

1. by cross-distance
- no cross distance; all in one
- by cross distance; all in unique
- by partial cross distance; within-sentence vs. cross sentence

2. relation format
- [CLS] S1 [SEP] S2 [SEP]
- [CLS] S1 S2 [SEP]

We only handle (1) in data generation here, (2) will be handled by the data_utils
"""
def to_tsv(data, fn):
    header = "\t".join([str(i+1) for i in range(len(data[0]))])
    with open(fn, "w") as f:
        f.write(f"{header}\n")
        for each in data:
            d = "\t".join([str(e) for e in each])
            f.write(f"{d}\n")


def to_5_cv(data, ofd):
    if not os.path.isdir(ofd):
        os.mkdir(ofd)
    
    np.random.seed(13)
    np.random.shuffle(data)
    
    dfs = np.array_split(data, 5)
    a = [0,1,2,3,4]
    for each in combinations(a, 4):
        b = list(set(a) - set(each))[0]
        n = dfs[b]
        m = []
        for k in each:
            m.extend(dfs[k])
        if not os.path.isdir(os.path.join(ofd, f"sample{b}")):
            os.mkdir(os.path.join(ofd, f"sample{b}"))
        
        to_tsv(m, os.path.join(ofd, f"sample{b}", "train.tsv"))
        to_tsv(n, os.path.join(ofd, f"sample{b}", "dev.tsv"))


def all_in_one(*dd, dn="2018n2c2", do_train=True):
    data = []
    for d in dd:
        for k, v in d.items():
            for each in v:
#                 data.append(each[1:])
                data.append(each)
    
    output_path = f"./data/{dn}_aio_th{CUTOFF}"
    p = Path(output_path)
    p.mkdir(parents=True, exist_ok=True)
    
    if do_train:
        to_tsv(data, p/"train.tsv")
#         to_5_cv(data, p.as_posix())
    else:
        to_tsv(data, p/"test.tsv")
    

def all_in_unique(*dd, dn="2018n2c2", do_train=True):
    for idx in range(CUTOFF+1):
        data = []
        for d in dd:
            for k, v in d.items():
                for each in v:
                    if k == idx:
                        data.append(each[1:])
        
        output_path = f"./data/{dn}_aiu_th{CUTOFF}"
        p = Path(output_path) / f"cutoff_{idx}"
        p.mkdir(parents=True, exist_ok=True)
        if do_train:
            to_tsv(data, p/"train.tsv")
#             to_5_cv(data, p.as_posix())
        else:
            to_tsv(data, p/"test.tsv")

            
def partial_unique(*dd, dn="2018n2c2", do_train=True):
    within = []
    cross = []
    
    for d in dd:
        for k, v in d.items():
            for each in v:
                if k == 0:
                    within.append(each[1:])
                else:
                    cross.append(each[1:])
    
    output_path = f"./data/{dn}_pu_th{CUTOFF}"
    p = Path(output_path)
    p1 = p / "within"
    p2 = p / "cross"
    p1.mkdir(parents=True, exist_ok=True)
    p2.mkdir(parents=True, exist_ok=True)
    
    if do_train:
        to_tsv(within, p1/"train.tsv")
#         to_5_cv(within, p1.as_posix())
        to_tsv(cross, p2/"train.tsv")
#         to_5_cv(cross, p2.as_posix())
    else:
        to_tsv(within, p1/"test.tsv")
        to_tsv(cross, p2/"test.tsv")

In [1]:
# create ann for test purpose
def extract_only_entity(input_path, output_path):
    pi = Path(input_path)
    po = Path(output_path)
    po.mkdir(exist_ok=True, parents=True)
    for fid in pi.glob("*.ann"):
        ofn = po / fid.name
        with open(fid, "r") as f1, open(ofn, "w") as f2:
            for line in f1.readlines():
                if line.startswith("T"):
                    f2.write(line)

In [12]:
# general pre-defined special tags
EN1_START = "[s1]"
EN1_END = "[e1]"
EN2_START = "[s2]"
EN2_END = "[e2]"
NEG_REL = "NonRel"
CUTOFF = 1

In [13]:
def _flat(dd):
    c = 0
    for k, v in dd.items():
        c += len(v)
    return c

In [2]:
train_dev_root ='/home/zehao.yu/workspace/py3/dr_ann/data/used/train_set'
test_root = '/home/zehao.yu/workspace/py3/dr_ann/data/used/test_set'

In [21]:
dr_valid = {
 'located', 'laterality-lesion', 'severity-lesion'
}

dr_valid_comb_en = {
    ('lesion', 'eye_part'), ('laterality', 'lesion'), ('severity', 'lesion')
}

In [30]:

d = {"nc":defaultdict(list), "dr": defaultdict(list)}

for i in range(1, 5, 1):
    CUTOFF=i
    dpos, dneg = create_training_samples(train_dev_root,dr_valid,dr_valid_comb_en)
    preds = create_test_samples(test_root,dr_valid,dr_valid_comb_en)
    d['nc'][CUTOFF].append((_flat(dpos), _flat(dneg), _flat(preds)))

    
for i in range(1, 5, 1):
    CUTOFF=i
    dpos, dneg = create_training_samples(train_dev_root,dr_valid,dr_valid_comb_en)
    all_in_one(dpos, dneg, dn="dr_relation", do_train=True)
    partial_unique(dpos, dneg, dn="dr_relation", do_train=True)
    all_in_unique(dpos, dneg, dn="dr_relation", do_train=True)
    
    preds = create_test_samples(test_root,dr_valid,dr_valid_comb_en)
    all_in_one(preds, dn="dr_relation", do_train=False)
    partial_unique(preds, dn="dr_relation", do_train=False)
    all_in_unique(preds, dn="dr_relation", do_train=False)
    d['dr'][CUTOFF].append((_flat(dpos), _flat(dneg), _flat(preds)))

2021-06-15 16:43:19,241 INFO sentence boundary detection class initiated.
2021-06-15 16:43:19,242 INFO word level tokenization with replace_number set to False
2021-06-15 16:43:19,242 INFO current processing /home/zehao.yu/workspace/py3/dr_ann/data/used/train_set/note_9152.txt ...
2021-06-15 16:43:19,318 INFO sentence boundary detection class initiated.
2021-06-15 16:43:19,318 INFO word level tokenization with replace_number set to False
2021-06-15 16:43:19,319 INFO current processing /home/zehao.yu/workspace/py3/dr_ann/data/used/train_set/note_863.txt ...
2021-06-15 16:43:19,370 INFO sentence boundary detection class initiated.
2021-06-15 16:43:19,371 INFO word level tokenization with replace_number set to False
2021-06-15 16:43:19,371 INFO current processing /home/zehao.yu/workspace/py3/dr_ann/data/used/train_set/note_4550.txt ...
2021-06-15 16:43:19,426 INFO sentence boundary detection class initiated.
2021-06-15 16:43:19,427 INFO word level tokenization with replace_number set to F

last index not match  ('MA', 'lesion', (799, 801))
last index not match  ('MA', 'lesion', (889, 891))


2021-06-15 16:43:19,480 INFO sentence boundary detection class initiated.
2021-06-15 16:43:19,481 INFO word level tokenization with replace_number set to False
2021-06-15 16:43:19,482 INFO current processing /home/zehao.yu/workspace/py3/dr_ann/data/used/train_set/1458921_note_3276.txt ...
2021-06-15 16:43:19,546 INFO sentence boundary detection class initiated.
2021-06-15 16:43:19,547 INFO word level tokenization with replace_number set to False
2021-06-15 16:43:19,548 INFO current processing /home/zehao.yu/workspace/py3/dr_ann/data/used/train_set/note_6884.txt ...
2021-06-15 16:43:19,601 INFO sentence boundary detection class initiated.
2021-06-15 16:43:19,602 INFO word level tokenization with replace_number set to False
2021-06-15 16:43:19,603 INFO current processing /home/zehao.yu/workspace/py3/dr_ann/data/used/train_set/910329_note_1460.txt ...
2021-06-15 16:43:19,654 INFO sentence boundary detection class initiated.
2021-06-15 16:43:19,655 INFO word level tokenization with replace

2021-06-15 16:43:21,201 INFO sentence boundary detection class initiated.
2021-06-15 16:43:21,201 INFO word level tokenization with replace_number set to False
2021-06-15 16:43:21,202 INFO current processing /home/zehao.yu/workspace/py3/dr_ann/data/used/train_set/note_1245.txt ...
2021-06-15 16:43:21,253 INFO sentence boundary detection class initiated.
2021-06-15 16:43:21,254 INFO word level tokenization with replace_number set to False
2021-06-15 16:43:21,255 INFO current processing /home/zehao.yu/workspace/py3/dr_ann/data/used/train_set/994046_note_1743.txt ...
2021-06-15 16:43:21,307 INFO sentence boundary detection class initiated.
2021-06-15 16:43:21,308 INFO word level tokenization with replace_number set to False
2021-06-15 16:43:21,308 INFO current processing /home/zehao.yu/workspace/py3/dr_ann/data/used/train_set/1162635_note_2414.txt ...
2021-06-15 16:43:21,366 INFO sentence boundary detection class initiated.
2021-06-15 16:43:21,367 INFO word level tokenization with replace

last index not match  ('MA', 'lesion', (1338, 1340))


...

2021-06-15 16:47:31,440 INFO sentence boundary detection class initiated.
2021-06-15 16:47:31,441 INFO word level tokenization with replace_number set to False
2021-06-15 16:47:31,441 INFO current processing /home/zehao.yu/workspace/py3/dr_ann/data/used/test_set/note_1533.txt ...
2021-06-15 16:47:31,511 INFO sentence boundary detection class initiated.
2021-06-15 16:47:31,512 INFO word level tokenization with replace_number set to False
2021-06-15 16:47:31,512 INFO current processing /home/zehao.yu/workspace/py3/dr_ann/data/used/test_set/note_8850.txt ...
2021-06-15 16:47:31,568 INFO sentence boundary detection class initiated.
2021-06-15 16:47:31,569 INFO word level tokenization with replace_number set to False
2021-06-15 16:47:31,569 INFO current processing /home/zehao.yu/workspace/py3/dr_ann/data/used/test_set/note_5884.txt ...
2021-06-15 16:47:31,626 INFO sentence boundary detection class initiated.
2021-06-15 16:47:31,627 INFO word level tokenization with replace_number set to Fal

last index not match  ('Right  ', 'laterality', (1346, 1353))
last index not match  ('Left  ', 'laterality', (2351, 2357))


2021-06-15 16:47:33,038 INFO sentence boundary detection class initiated.
2021-06-15 16:47:33,039 INFO word level tokenization with replace_number set to False
2021-06-15 16:47:33,039 INFO current processing /home/zehao.yu/workspace/py3/dr_ann/data/used/test_set/note_2444.txt ...
2021-06-15 16:47:33,041 WARNING '7.2 ' cannot be parsed by current rule.
2021-06-15 16:47:33,099 INFO sentence boundary detection class initiated.
2021-06-15 16:47:33,100 INFO word level tokenization with replace_number set to False
2021-06-15 16:47:33,100 INFO current processing /home/zehao.yu/workspace/py3/dr_ann/data/used/test_set/1424866_note_3176.txt ...
2021-06-15 16:47:33,155 INFO sentence boundary detection class initiated.
2021-06-15 16:47:33,156 INFO word level tokenization with replace_number set to False
2021-06-15 16:47:33,157 INFO current processing /home/zehao.yu/workspace/py3/dr_ann/data/used/test_set/1936668_note_5255.txt ...
2021-06-15 16:47:33,215 INFO sentence boundary detection class initi

last index not match  ('severe\xa0', 'severity', (1734, 1741))
last index not match  ('OD\xa0', 'laterality', (1987, 1990))
last index not match  ('OS\xa0', 'laterality', (2032, 2035))
last index not match  ('MA', 'lesion', (450, 452))


2021-06-15 16:47:33,724 INFO sentence boundary detection class initiated.
2021-06-15 16:47:33,725 INFO word level tokenization with replace_number set to False
2021-06-15 16:47:33,726 INFO current processing /home/zehao.yu/workspace/py3/dr_ann/data/used/test_set/2476700_note_7181.txt ...
2021-06-15 16:47:33,780 INFO sentence boundary detection class initiated.
2021-06-15 16:47:33,781 INFO word level tokenization with replace_number set to False
2021-06-15 16:47:33,782 INFO current processing /home/zehao.yu/workspace/py3/dr_ann/data/used/test_set/note_3956.txt ...


In [39]:
dpos[0]


[(0,
  'laterality-lesion',
  'Proliferative diabetic retinopathy of [s1] right eye [e1] with macular edema determined by examination associated with type 2 diabetes mellitus E11 . 3511 250.50 362.07 362.02 Treatment Hx : IVA OS 3 / 6 / 18 PRP OD ( unknown date ) Intravitreal injection ( unknown # and date ) NS in both eyes .',
  '[s2] Proliferative diabetic retinopathy [e2] of right eye with macular edema determined by examination associated with type 2 diabetes mellitus E11 . 3511 250.50 362.07 362.02 Treatment Hx : IVA OS 3 / 6 / 18 PRP OD ( unknown date ) Intravitreal injection ( unknown # and date ) NS in both eyes .',
  'laterality',
  'lesion',
  'T12',
  'T1',
  'note_9152'),
 (0,
  'laterality-lesion',
  'Proliferative diabetic retinopathy of [s1] right eye [e1] with macular edema determined by examination associated with type 2 diabetes mellitus E11 . 3511 250.50 362.07 362.02 Treatment Hx : IVA OS 3 / 6 / 18 PRP OD ( unknown date ) Intravitreal injection ( unknown # and date

In [ ]:
"""
prev:
n2c2: aio_th1 108147 71016
umass: aio_th1 102752 16035
"""

"""
binary:
{'nc': defaultdict(list,
             {1: [(34790, 69403, 69395)],
              2: [(35239, 122103, 103058)],
              3: [(35401, 172013, 135066)],
              4: [(35489, 220521, 165704)]}),
 'um': defaultdict(list,
             {1: [(21254, 75850, 15410)],
              2: [(21926, 100578, 19320)],
              3: [(22184, 122651, 22789)],
              4: [(22316, 144038, 26150)]})}

class:
{'nc': defaultdict(list,
             {1: [(34790, 69403, 69395)],
              2: [(35239, 122103, 103058)],
              3: [(35401, 172013, 135066)],
              4: [(35489, 220521, 165704)]}),
 'um': defaultdict(list,
             {1: [(21254, 75850, 15410)],
              2: [(21926, 100578, 19320)],
              3: [(22184, 122651, 22789)],
              4: [(22316, 144038, 26150)]})}

"""

## n2c2

In [12]:
n2c2_training = "/Users/alexgre/workspace/data/2018_n2c2_ade/track2-training_data/"
n2c2_test = "/Users/alexgre/workspace/data/2018_n2c2_ade/gold_standard_test/"
n2c2_entity_only = "/Users/alexgre/workspace/data/2018_n2c2_ade/gold_standard_test_only_entity"

In [13]:
n2c2_valid = {'Route-Drug', 'Reason-Drug', 'ADE-Drug', 'Frequency-Drug',
              'Strength-Drug', 'Dosage-Drug', 'Drug-Drug', 'Form-Drug', 'Duration-Drug',
              'Not-Rel'}

n2c2_valid_comb = {
    ('ADE', 'Drug'), ('Reason', 'Drug'),
    ('Strength', 'Drug'), ('Route', 'Drug'), 
    ('Frequency', 'Drug'), ('Dosage', 'Drug'),
    ('Form', 'Drug'), ('Duration', 'Drug')
}

In [14]:
entp2rel = {
    ('ADE', 'Drug'):'ADE-Drug', 
    ('Reason', 'Drug'):'Reason-Drug',
    ('Strength', 'Drug'):'Strength-Drug', 
    ('Route', 'Drug'):'Route-Drug', 
    ('Frequency', 'Drug'):'Frequency-Drug', 
    ('Dosage', 'Drug'):'Dosage-Drug',
    ('Form', 'Drug'):'Form-Drug', 
    ('Duration', 'Drug'):'Duration-Drug'
}

entp2rel, entp2rel[('ADE', 'Drug')]

({('ADE', 'Drug'): 'ADE-Drug',
  ('Reason', 'Drug'): 'Reason-Drug',
  ('Strength', 'Drug'): 'Strength-Drug',
  ('Route', 'Drug'): 'Route-Drug',
  ('Frequency', 'Drug'): 'Frequency-Drug',
  ('Dosage', 'Drug'): 'Dosage-Drug',
  ('Form', 'Drug'): 'Form-Drug',
  ('Duration', 'Drug'): 'Duration-Drug'},
 'ADE-Drug')

In [ ]:
pkl_save(entp2rel, "../data/2018n2c2_relation_processed/binary/2018n2c2_mapping.pkl")

In [ ]:
extract_only_entity(n2c2_test, n2c2_entity_only)

### training

In [39]:
CUTOFF = 1

In [21]:
rel2idx, idx2rel, rel_cnts = get_all_rel_types(n2c2_training, n2c2_valid_comb)
rel2idx, idx2rel, rel_cnts, reduce(lambda x, y: x + y, [e[1] for e in rel_cnts])

NameError: name 'get_all_rel_types' is not defined

In [40]:
dpos, dneg = create_training_samples(n2c2_training, n2c2_valid, n2c2_valid_comb)

2021-05-01 11:23:45,431 INFO sentence boundary detection class initiated.
2021-05-01 11:23:45,431 INFO word level tokenization with replace_number set to False
2021-05-01 11:23:45,432 INFO current processing /Users/alexgre/workspace/data/2018_n2c2_ade/track2-training_data/110727.txt ...
2021-05-01 11:23:45,478 INFO qSunday
2021-05-01 11:23:45,479 WARNING 'qSunday' => 'q' 'Sunday'
2021-05-01 11:23:45,488 INFO MSContin
2021-05-01 11:23:45,489 WARNING 'MSContin' => 'MS' 'Contin'
2021-05-01 11:23:45,526 WARNING ('3', 'Dosage', (12227, 12228)) offset is overlapped with previous entity; current tok not overlap


first index not match  ('3', 'Dosage', (12227, 12228))
last index not match  ('3', 'Dosage', (12227, 12228))


2021-05-01 11:23:46,676 INFO sentence boundary detection class initiated.
2021-05-01 11:23:46,676 INFO word level tokenization with replace_number set to False
2021-05-01 11:23:46,677 INFO current processing /Users/alexgre/workspace/data/2018_n2c2_ade/track2-training_data/101372.txt ...
2021-05-01 11:23:46,697 INFO AnGap
2021-05-01 11:23:46,697 WARNING 'AnGap' => 'An' 'Gap'
2021-05-01 11:23:46,698 INFO AlkPhos
2021-05-01 11:23:46,698 WARNING 'AlkPhos' => 'Alk' 'Phos'
2021-05-01 11:23:46,699 INFO TotBili
2021-05-01 11:23:46,699 WARNING 'TotBili' => 'Tot' 'Bili'
2021-05-01 11:23:47,132 INFO sentence boundary detection class initiated.
2021-05-01 11:23:47,133 INFO word level tokenization with replace_number set to False
2021-05-01 11:23:47,133 INFO current processing /Users/alexgre/workspace/data/2018_n2c2_ade/track2-training_data/106384.txt ...
2021-05-01 11:23:47,155 INFO LDLcalc
2021-05-01 11:23:47,156 WARNING 'LDLcalc' => 'LD' 'Lcalc'
2021-05-01 11:23:47,162 INFO NameIs
2021-05-01 11:

last index not match  ('b.i.d', 'Frequency', (8199, 8204))
last index not match  ('b.i.d', 'Frequency', (8300, 8305))


2021-05-01 11:23:47,464 INFO sentence boundary detection class initiated.
2021-05-01 11:23:47,465 INFO word level tokenization with replace_number set to False
2021-05-01 11:23:47,465 INFO current processing /Users/alexgre/workspace/data/2018_n2c2_ade/track2-training_data/110445.txt ...
2021-05-01 11:23:47,495 WARNING ('[**Doctor First Name **]', 'Drug', (5177, 5201)) offset is overlapped with previous entity; current tok not overlap
2021-05-01 11:23:47,577 INFO sentence boundary detection class initiated.
2021-05-01 11:23:47,578 INFO word level tokenization with replace_number set to False
2021-05-01 11:23:47,578 INFO current processing /Users/alexgre/workspace/data/2018_n2c2_ade/track2-training_data/115267.txt ...
2021-05-01 11:23:47,606 INFO AnGap
2021-05-01 11:23:47,607 WARNING 'AnGap' => 'An' 'Gap'
2021-05-01 11:23:47,608 INFO AlkPhos
2021-05-01 11:23:47,608 WARNING 'AlkPhos' => 'Alk' 'Phos'
2021-05-01 11:23:47,609 INFO TotBili
2021-05-01 11:23:47,609 WARNING 'TotBili' => 'Tot' 'B

...

last index not match  ('2L', 'Dosage', (11456, 11458))
first index not match  ('NC', 'Route', (11458, 11460))
last index not match  ('NC', 'Route', (11458, 11460))
last index not match  ('2L', 'Dosage', (11665, 11667))
first index not match  ('NC', 'Route', (11667, 11669))
last index not match  ('NC', 'Route', (11667, 11669))


2021-05-01 11:25:35,147 INFO sentence boundary detection class initiated.
2021-05-01 11:25:35,151 INFO word level tokenization with replace_number set to False
2021-05-01 11:25:35,152 INFO current processing /Users/alexgre/workspace/data/2018_n2c2_ade/track2-training_data/117907.txt ...
2021-05-01 11:25:35,168 INFO AnGap
2021-05-01 11:25:35,169 WARNING 'AnGap' => 'An' 'Gap'
2021-05-01 11:25:35,170 INFO AnGap
2021-05-01 11:25:35,170 WARNING 'AnGap' => 'An' 'Gap'
2021-05-01 11:25:35,171 INFO AnGap
2021-05-01 11:25:35,172 WARNING 'AnGap' => 'An' 'Gap'
2021-05-01 11:25:35,172 INFO AlkPhos
2021-05-01 11:25:35,173 WARNING 'AlkPhos' => 'Alk' 'Phos'
2021-05-01 11:25:35,173 INFO TotBili
2021-05-01 11:25:35,173 WARNING 'TotBili' => 'Tot' 'Bili'
2021-05-01 11:25:35,179 INFO freeCa
2021-05-01 11:25:35,180 WARNING 'freeCa' => 'free' 'Ca'
2021-05-01 11:25:35,262 INFO sentence boundary detection class initiated.
2021-05-01 11:25:35,263 INFO word level tokenization with replace_number set to False
202

first index not match  ('1', 'Dosage', (9745, 9746))
last index not match  ('1', 'Dosage', (9745, 9746))
first index not match  ('2', 'Dosage', (9727, 9728))
last index not match  ('2', 'Dosage', (9727, 9728))
first index not match  ('1', 'Dosage', (10061, 10062))
last index not match  ('1', 'Dosage', (10061, 10062))


2021-05-01 11:25:36,368 INFO sentence boundary detection class initiated.
2021-05-01 11:25:36,368 INFO word level tokenization with replace_number set to False
2021-05-01 11:25:36,369 INFO current processing /Users/alexgre/workspace/data/2018_n2c2_ade/track2-training_data/119069.txt ...
2021-05-01 11:25:36,653 INFO sentence boundary detection class initiated.
2021-05-01 11:25:36,653 INFO word level tokenization with replace_number set to False
2021-05-01 11:25:36,654 INFO current processing /Users/alexgre/workspace/data/2018_n2c2_ade/track2-training_data/104979.txt ...
2021-05-01 11:25:36,811 INFO sentence boundary detection class initiated.
2021-05-01 11:25:36,812 INFO word level tokenization with replace_number set to False
2021-05-01 11:25:36,812 INFO current processing /Users/alexgre/workspace/data/2018_n2c2_ade/track2-training_data/105050.txt ...
2021-05-01 11:25:36,848 INFO NotDone
2021-05-01 11:25:36,849 WARNING 'NotDone' => 'Not' 'Done'
2021-05-01 11:25:36,913 WARNING ('morphin

last index not match  ('b.i.d', 'Frequency', (2293, 2298))


In [25]:
"""
0 32436
1 2409
3 162
2 446
4 94

3 51321
1 55317
4 49820
2 54012
0 18137
"""

for k, v in dpos.items():
    print(k, len(v))
    
for k, v in dneg.items():
    print(k, len(v))

0 32302
1 2464
1 53586
0 15919


In [41]:
dpos[1]

[(1,
  'Frequency-Drug',
  'Hospital 1 [s1] ( 2 times a day ) as needed [e1] for itchiness : Can apply to back of calf or other regions of itchy skin .',
  'Disp : * 60 Capsule ( s ) * Refills : * 0 * 22 . [s2] Camphor - Menthol [e2] 0.5 - 0.5 % Lotion Sig : One ( 1 ) Appl Topical',
  'Frequency',
  'Drug',
  'T3',
  'T1',
  '110727'),
 (1,
  'Frequency-Drug',
  'Subcutaneous [s1] once a month [e1] .',
  'Disp : * 30 Tablet ( s ) * Refills : * 0 * 5 . [s2] Golimumab [e2] 50 mg / 0.5 mL Pen Injector Sig : Fifty ( 50 ) mg',
  'Frequency',
  'Drug',
  'T278',
  'T178',
  '110727'),
 (1,
  'Route-Drug',
  '[s1] Subcutaneous [e1] once a month .',
  'Disp : * 30 Tablet ( s ) * Refills : * 0 * 5 . [s2] Golimumab [e2] 50 mg / 0.5 mL Pen Injector Sig : Fifty ( 50 ) mg',
  'Route',
  'Drug',
  'T277',
  'T178',
  '110727'),
 (1,
  'Reason-Drug',
  'Sig : 1 - 2 Tablets PO every six ( 6 ) hours as needed for [s1] heartburn [e1] .',
  '6 . [s2] Phenobarb - Hyoscy - Atropine - Scop [e2] 16.2 - 0.103

In [42]:
all_in_one(dpos, dneg, dn="2018n2c2_gs_aio_1_train", do_train=True)

In [ ]:
all_in_one(dpos, dneg, dn="2018n2c2", do_train=True)
partial_unique(dpos, dneg, dn="2018n2c2", do_train=True)
all_in_unique(dpos, dneg, dn="2018n2c2", do_train=True)

### test

In [ ]:
_, _, rel_cnts = get_all_rel_types(n2c2_test, n2c2_valid_comb)
rel_cnts, reduce(lambda x, y: x + y, [e[1] for e in rel_cnts])

In [ ]:
preds = create_test_samples(n2c2_test, n2c2_valid, n2c2_valid_comb)

In [ ]:
preds[0]

In [ ]:
"""
0 34231
1 36929
2 34477
3 32600
4 31398
"""
for k, v in preds.items():
    print(k, len(v))

In [ ]:
all_in_one(preds, dn="2018n2c2", do_train=False)
partial_unique(preds, dn="2018n2c2", do_train=False)
all_in_unique(preds, dn="2018n2c2", do_train=False)

### test with labels

In [35]:
CUTOFF = 4

In [36]:
dpos, dneg = create_training_samples(n2c2_test, n2c2_valid, n2c2_valid_comb)

2021-04-30 18:41:39,561 INFO sentence boundary detection class initiated.
2021-04-30 18:41:39,561 INFO word level tokenization with replace_number set to False
2021-04-30 18:41:39,562 INFO current processing /Users/alexgre/workspace/data/2018_n2c2_ade/gold_standard_test/107515.txt ...
2021-04-30 18:41:39,583 INFO freeCa
2021-04-30 18:41:39,583 WARNING 'freeCa' => 'free' 'Ca'
2021-04-30 18:41:39,622 WARNING ('hypergylcemia', 'ADE', (9527, 9540)) offset is overlapped with previous entity; current tok not overlap
2021-04-30 18:41:40,280 INFO sentence boundary detection class initiated.
2021-04-30 18:41:40,280 INFO word level tokenization with replace_number set to False
2021-04-30 18:41:40,281 INFO current processing /Users/alexgre/workspace/data/2018_n2c2_ade/gold_standard_test/155521.txt ...
2021-04-30 18:41:40,300 INFO AnGap
2021-04-30 18:41:40,301 WARNING 'AnGap' => 'An' 'Gap'
2021-04-30 18:41:40,302 INFO AlkPhos
2021-04-30 18:41:40,302 WARNING 'AlkPhos' => 'Alk' 'Phos'
2021-04-30 18:

last index not match  ('p.o', 'Route', (9048, 9051))


2021-04-30 18:41:41,202 INFO sentence boundary detection class initiated.
2021-04-30 18:41:41,203 INFO word level tokenization with replace_number set to False
2021-04-30 18:41:41,203 INFO current processing /Users/alexgre/workspace/data/2018_n2c2_ade/gold_standard_test/139955.txt ...
2021-04-30 18:41:41,226 INFO AnGap
2021-04-30 18:41:41,227 WARNING 'AnGap' => 'An' 'Gap'
2021-04-30 18:41:41,227 INFO AnGap
2021-04-30 18:41:41,228 WARNING 'AnGap' => 'An' 'Gap'
2021-04-30 18:41:41,229 INFO AnGap
2021-04-30 18:41:41,229 WARNING 'AnGap' => 'An' 'Gap'
2021-04-30 18:41:41,230 INFO AnGap
2021-04-30 18:41:41,230 WARNING 'AnGap' => 'An' 'Gap'
2021-04-30 18:41:41,231 INFO AnGap
2021-04-30 18:41:41,231 WARNING 'AnGap' => 'An' 'Gap'
2021-04-30 18:41:41,232 INFO AnGap
2021-04-30 18:41:41,233 WARNING 'AnGap' => 'An' 'Gap'
2021-04-30 18:41:41,233 INFO AnGap
2021-04-30 18:41:41,234 WARNING 'AnGap' => 'An' 'Gap'
2021-04-30 18:41:41,234 INFO AnGap
2021-04-30 18:41:41,235 WARNING 'AnGap' => 'An' 'Gap'
20

last index not match  ('1L', 'Dosage', (936, 938))
first index not match  ('NS', 'Drug', (938, 940))
last index not match  ('NS', 'Drug', (938, 940))


2021-04-30 18:41:42,137 INFO sentence boundary detection class initiated.
2021-04-30 18:41:42,138 INFO word level tokenization with replace_number set to False
2021-04-30 18:41:42,138 INFO current processing /Users/alexgre/workspace/data/2018_n2c2_ade/gold_standard_test/104140.txt ...
2021-04-30 18:41:42,158 INFO AnGap
2021-04-30 18:41:42,159 WARNING 'AnGap' => 'An' 'Gap'
2021-04-30 18:41:42,162 INFO AnGap
2021-04-30 18:41:42,163 WARNING 'AnGap' => 'An' 'Gap'
2021-04-30 18:41:42,163 INFO AnGap
2021-04-30 18:41:42,164 WARNING 'AnGap' => 'An' 'Gap'
2021-04-30 18:41:42,165 INFO AnGap
2021-04-30 18:41:42,165 WARNING 'AnGap' => 'An' 'Gap'
2021-04-30 18:41:42,166 INFO AnGap
2021-04-30 18:41:42,166 WARNING 'AnGap' => 'An' 'Gap'
2021-04-30 18:41:42,167 INFO AlkPhos
2021-04-30 18:41:42,167 WARNING 'AlkPhos' => 'Alk' 'Phos'
2021-04-30 18:41:42,168 INFO TotBili
2021-04-30 18:41:42,168 WARNING 'TotBili' => 'Tot' 'Bili'
2021-04-30 18:41:42,487 INFO sentence boundary detection class initiated.
2021-

...

last index not match  ('p.o', 'Route', (5493, 5496))
last index not match  ('p.o', 'Route', (5555, 5558))
last index not match  ('p.o', 'Route', (5625, 5628))
last index not match  ('p.o', 'Route', (6209, 6212))


2021-04-30 18:42:47,351 INFO sentence boundary detection class initiated.
2021-04-30 18:42:47,352 INFO word level tokenization with replace_number set to False
2021-04-30 18:42:47,352 INFO current processing /Users/alexgre/workspace/data/2018_n2c2_ade/gold_standard_test/198207.txt ...
2021-04-30 18:42:47,543 INFO sentence boundary detection class initiated.
2021-04-30 18:42:47,544 INFO word level tokenization with replace_number set to False
2021-04-30 18:42:47,544 INFO current processing /Users/alexgre/workspace/data/2018_n2c2_ade/gold_standard_test/113222.txt ...
2021-04-30 18:42:47,570 INFO AnGap
2021-04-30 18:42:47,570 WARNING 'AnGap' => 'An' 'Gap'
2021-04-30 18:42:47,583 INFO NameIs
2021-04-30 18:42:47,584 WARNING 'NameIs' => 'Name' 'Is'
2021-04-30 18:42:47,585 INFO NameIs
2021-04-30 18:42:47,586 WARNING 'NameIs' => 'Name' 'Is'
2021-04-30 18:42:47,586 INFO NameIs
2021-04-30 18:42:47,587 WARNING 'NameIs' => 'Name' 'Is'
2021-04-30 18:42:47,607 WARNING ('pressure dropped', 'Reason', 

first index not match  ('1', 'Dosage', (1583, 1584))
last index not match  ('1', 'Dosage', (1583, 1584))
first index not match  ('1', 'Dosage', (2128, 2129))
last index not match  ('1', 'Dosage', (2128, 2129))
first index not match  ('3', 'Dosage', (8340, 8341))
last index not match  ('3', 'Dosage', (8340, 8341))


2021-04-30 18:42:50,942 INFO sentence boundary detection class initiated.
2021-04-30 18:42:50,943 INFO word level tokenization with replace_number set to False
2021-04-30 18:42:50,943 INFO current processing /Users/alexgre/workspace/data/2018_n2c2_ade/gold_standard_test/104549.txt ...
2021-04-30 18:42:50,977 INFO AnGap
2021-04-30 18:42:50,977 WARNING 'AnGap' => 'An' 'Gap'
2021-04-30 18:42:50,980 INFO AnGap
2021-04-30 18:42:50,981 WARNING 'AnGap' => 'An' 'Gap'
2021-04-30 18:42:50,981 INFO AnGap
2021-04-30 18:42:50,982 WARNING 'AnGap' => 'An' 'Gap'
2021-04-30 18:42:50,983 INFO AnGap
2021-04-30 18:42:50,983 WARNING 'AnGap' => 'An' 'Gap'
2021-04-30 18:42:50,984 INFO AnGap
2021-04-30 18:42:50,985 WARNING 'AnGap' => 'An' 'Gap'
2021-04-30 18:42:50,986 INFO AnGap
2021-04-30 18:42:50,986 WARNING 'AnGap' => 'An' 'Gap'
2021-04-30 18:42:50,986 INFO AlkPhos
2021-04-30 18:42:50,987 WARNING 'AlkPhos' => 'Alk' 'Phos'
2021-04-30 18:42:50,987 INFO TotBili
2021-04-30 18:42:50,988 WARNING 'TotBili' => 'To

first index not match  ('in', 'Reason', (4691, 4693))
last index not match  ('in', 'Reason', (4691, 4693))


In [37]:
all_in_one(dpos, dneg, dn="2018n2c2_gs", do_train=False)

In [43]:
for CUTOFF in range(1,5):
    dpos, dneg = create_training_samples(n2c2_test, n2c2_valid, n2c2_valid_comb)
    all_in_one(dpos, dneg, dn="2018n2c2_gs", do_train=False)

2021-05-01 11:26:29,219 INFO sentence boundary detection class initiated.
2021-05-01 11:26:29,220 INFO word level tokenization with replace_number set to False
2021-05-01 11:26:29,220 INFO current processing /Users/alexgre/workspace/data/2018_n2c2_ade/gold_standard_test/107515.txt ...
2021-05-01 11:26:29,242 INFO freeCa
2021-05-01 11:26:29,242 WARNING 'freeCa' => 'free' 'Ca'
2021-05-01 11:26:29,282 WARNING ('hypergylcemia', 'ADE', (9527, 9540)) offset is overlapped with previous entity; current tok not overlap
2021-05-01 11:26:29,942 INFO sentence boundary detection class initiated.
2021-05-01 11:26:29,943 INFO word level tokenization with replace_number set to False
2021-05-01 11:26:29,943 INFO current processing /Users/alexgre/workspace/data/2018_n2c2_ade/gold_standard_test/155521.txt ...
2021-05-01 11:26:29,963 INFO AnGap
2021-05-01 11:26:29,963 WARNING 'AnGap' => 'An' 'Gap'
2021-05-01 11:26:29,964 INFO AlkPhos
2021-05-01 11:26:29,964 WARNING 'AlkPhos' => 'Alk' 'Phos'
2021-05-01 11:

last index not match  ('p.o', 'Route', (9048, 9051))


2021-05-01 11:26:30,876 INFO sentence boundary detection class initiated.
2021-05-01 11:26:30,877 INFO word level tokenization with replace_number set to False
2021-05-01 11:26:30,877 INFO current processing /Users/alexgre/workspace/data/2018_n2c2_ade/gold_standard_test/139955.txt ...
2021-05-01 11:26:30,899 INFO AnGap
2021-05-01 11:26:30,900 WARNING 'AnGap' => 'An' 'Gap'
2021-05-01 11:26:30,901 INFO AnGap
2021-05-01 11:26:30,901 WARNING 'AnGap' => 'An' 'Gap'
2021-05-01 11:26:30,902 INFO AnGap
2021-05-01 11:26:30,903 WARNING 'AnGap' => 'An' 'Gap'
2021-05-01 11:26:30,904 INFO AnGap
2021-05-01 11:26:30,904 WARNING 'AnGap' => 'An' 'Gap'
2021-05-01 11:26:30,905 INFO AnGap
2021-05-01 11:26:30,905 WARNING 'AnGap' => 'An' 'Gap'
2021-05-01 11:26:30,906 INFO AnGap
2021-05-01 11:26:30,906 WARNING 'AnGap' => 'An' 'Gap'
2021-05-01 11:26:30,907 INFO AnGap
2021-05-01 11:26:30,908 WARNING 'AnGap' => 'An' 'Gap'
2021-05-01 11:26:30,908 INFO AnGap
2021-05-01 11:26:30,909 WARNING 'AnGap' => 'An' 'Gap'
20

last index not match  ('1L', 'Dosage', (936, 938))
first index not match  ('NS', 'Drug', (938, 940))
last index not match  ('NS', 'Drug', (938, 940))


2021-05-01 11:26:31,830 INFO sentence boundary detection class initiated.
2021-05-01 11:26:31,830 INFO word level tokenization with replace_number set to False
2021-05-01 11:26:31,830 INFO current processing /Users/alexgre/workspace/data/2018_n2c2_ade/gold_standard_test/104140.txt ...
2021-05-01 11:26:31,851 INFO AnGap
2021-05-01 11:26:31,851 WARNING 'AnGap' => 'An' 'Gap'
2021-05-01 11:26:31,855 INFO AnGap
2021-05-01 11:26:31,855 WARNING 'AnGap' => 'An' 'Gap'
2021-05-01 11:26:31,856 INFO AnGap
2021-05-01 11:26:31,857 WARNING 'AnGap' => 'An' 'Gap'
2021-05-01 11:26:31,857 INFO AnGap
2021-05-01 11:26:31,858 WARNING 'AnGap' => 'An' 'Gap'
2021-05-01 11:26:31,858 INFO AnGap
2021-05-01 11:26:31,859 WARNING 'AnGap' => 'An' 'Gap'
2021-05-01 11:26:31,859 INFO AlkPhos
2021-05-01 11:26:31,860 WARNING 'AlkPhos' => 'Alk' 'Phos'
2021-05-01 11:26:31,860 INFO TotBili
2021-05-01 11:26:31,861 WARNING 'TotBili' => 'Tot' 'Bili'
2021-05-01 11:26:32,216 INFO sentence boundary detection class initiated.
2021-

...

last index not match  ('p.o', 'Route', (5493, 5496))
last index not match  ('p.o', 'Route', (5555, 5558))
last index not match  ('p.o', 'Route', (5625, 5628))
last index not match  ('p.o', 'Route', (6209, 6212))


2021-05-01 11:31:12,646 INFO sentence boundary detection class initiated.
2021-05-01 11:31:12,647 INFO word level tokenization with replace_number set to False
2021-05-01 11:31:12,647 INFO current processing /Users/alexgre/workspace/data/2018_n2c2_ade/gold_standard_test/198207.txt ...
2021-05-01 11:31:12,836 INFO sentence boundary detection class initiated.
2021-05-01 11:31:12,837 INFO word level tokenization with replace_number set to False
2021-05-01 11:31:12,837 INFO current processing /Users/alexgre/workspace/data/2018_n2c2_ade/gold_standard_test/113222.txt ...
2021-05-01 11:31:12,862 INFO AnGap
2021-05-01 11:31:12,863 WARNING 'AnGap' => 'An' 'Gap'
2021-05-01 11:31:12,875 INFO NameIs
2021-05-01 11:31:12,876 WARNING 'NameIs' => 'Name' 'Is'
2021-05-01 11:31:12,877 INFO NameIs
2021-05-01 11:31:12,877 WARNING 'NameIs' => 'Name' 'Is'
2021-05-01 11:31:12,878 INFO NameIs
2021-05-01 11:31:12,878 WARNING 'NameIs' => 'Name' 'Is'
2021-05-01 11:31:12,900 WARNING ('pressure dropped', 'Reason', 

first index not match  ('1', 'Dosage', (1583, 1584))
last index not match  ('1', 'Dosage', (1583, 1584))
first index not match  ('1', 'Dosage', (2128, 2129))
last index not match  ('1', 'Dosage', (2128, 2129))
first index not match  ('3', 'Dosage', (8340, 8341))
last index not match  ('3', 'Dosage', (8340, 8341))


2021-05-01 11:31:16,174 INFO sentence boundary detection class initiated.
2021-05-01 11:31:16,175 INFO word level tokenization with replace_number set to False
2021-05-01 11:31:16,175 INFO current processing /Users/alexgre/workspace/data/2018_n2c2_ade/gold_standard_test/104549.txt ...
2021-05-01 11:31:16,212 INFO AnGap
2021-05-01 11:31:16,212 WARNING 'AnGap' => 'An' 'Gap'
2021-05-01 11:31:16,216 INFO AnGap
2021-05-01 11:31:16,217 WARNING 'AnGap' => 'An' 'Gap'
2021-05-01 11:31:16,218 INFO AnGap
2021-05-01 11:31:16,218 WARNING 'AnGap' => 'An' 'Gap'
2021-05-01 11:31:16,219 INFO AnGap
2021-05-01 11:31:16,219 WARNING 'AnGap' => 'An' 'Gap'
2021-05-01 11:31:16,220 INFO AnGap
2021-05-01 11:31:16,220 WARNING 'AnGap' => 'An' 'Gap'
2021-05-01 11:31:16,221 INFO AnGap
2021-05-01 11:31:16,221 WARNING 'AnGap' => 'An' 'Gap'
2021-05-01 11:31:16,222 INFO AlkPhos
2021-05-01 11:31:16,222 WARNING 'AlkPhos' => 'Alk' 'Phos'
2021-05-01 11:31:16,223 INFO TotBili
2021-05-01 11:31:16,223 WARNING 'TotBili' => 'To

first index not match  ('in', 'Reason', (4691, 4693))
last index not match  ('in', 'Reason', (4691, 4693))


In [28]:
all_in_unique(dpos, dneg, dn="2018n2c2_gs", do_train=False)

## umass

In [ ]:
umass_valid = {
 'adverse', 'do', 'du', 'fr', 'reason', 'severity_type', 'Not-Rel', 'manner/route'
}

umass_valid_comb_en = {
    ('Drug', 'Dose'), ('Drug', 'Duration'), ('Drug', 'Frequency'), ('Drug', 'Route'),
    ('Drug', 'ADE'), ('Drug', 'Indication'), ('SSLIF', 'Severity'), ('ADE', 'Severity'),
    ('Indication', 'Severity')
}

In [ ]:
entp2rel = {
    ('Drug', 'Dose'): 'do', 
    ('Drug', 'Duration'): 'du', 
    ('Drug', 'Frequency'): 'fr', 
    ('Drug', 'Route'): 'manner/route',
    ('Drug', 'ADE'): 'adverse', 
    ('Drug', 'Indication'): 'reason', 
    ('SSLIF', 'Severity'): 'severity_type', 
    ('ADE', 'Severity'): 'severity_type',
    ('Indication', 'Severity'): 'severity_type'
}

entp2rel, entp2rel[('Drug', 'Indication')]

In [ ]:
pkl_save(entp2rel, "../data/2018umass_relation_processed/binary/2018umass_mapping.pkl")

In [ ]:
umass_training = "/Users/alexgre/workspace/data/2018_umass_ade/training/brat/"
umass_test = "/Users/alexgre/workspace/data/2018_umass_ade/test/brat/"
umass_entity_only = "/Users/alexgre/workspace/data/2018_umass_ade/test/brat_only_entity/"

In [ ]:
extract_only_entity(umass_test, umass_entity_only)

In [ ]:
total = []
n = []
for fn in Path(umass_training).glob("*.ann"):
    ners, rels = read_brat(fn)
    total.extend(rels)
    n.extend(ners)
ur = set([e[1] for e in total])
un = set([e[1] for e in n])
ur, un, len(total), len(n)

### training

In [ ]:
dpos, dneg = create_training_samples(umass_training, umass_valid, umass_valid_comb_en)

In [ ]:
"""
0 19594
1 2270
2 667
3 255
4 130

1 34698
4 21366
3 21857
2 25937
0 46510
"""

for k, v in dpos.items():
    print(k, len(v))

print()
    
for k, v in dneg.items():
    print(k, len(v))

In [ ]:
dpos[0][:3], dneg[0][:3]

In [ ]:
all_in_one(dpos, dneg, dn="2018umass", do_train=True)
partial_unique(dpos, dneg, dn="2018umass", do_train=True)
all_in_unique(dpos, dneg, dn="2018umass", do_train=True)

### test

In [ ]:
preds = create_test_samples(umass_test, umass_valid, umass_valid_comb_en)

In [ ]:
rel_mappings

In [ ]:
"""
0 3648
1 304
2 114
3 56
4 9
0 5200
4 1505
2 1653
1 2401
3 1524
"""

for k, v in preds.items():
    print(k, len(v))

In [ ]:
preds[0][:2]

In [ ]:
all_in_one(preds, dn="2018umass", do_train=False)
partial_unique(preds, dn="2018umass", do_train=False)
all_in_unique(preds, dn="2018umass", do_train=False)

## DDI

In [ ]:
ddi_training = "../data/DDICorpusBrat/Train/combine/"
ddi_test = "../data/DDICorpusBrat/Test/combine/"
ddi_entity_only = "../data/DDICorpusBrat/Test/ddi_entity_only/"
Path(ddi_entity_only).mkdir(parents=True, exist_ok=True)

In [ ]:
extract_only_entity(ddi_test, ddi_entity_only)

In [ ]:
total = []
n = []
for fn in Path(ddi_training).glob("*.ann"):
    ners, rels = read_brat(fn)
    total.extend(rels)
    n.extend(ners)
ur = set([e[1] for e in total])
un = set([e[1] for e in n])
ur, un, len(total), len(n)

In [ ]:
total = []
n = []
for fn in Path(ddi_test).glob("*.ann"):
    ners, rels = read_brat(fn)
    total.extend(rels)
    n.extend(ners)
ur = set([e[1] for e in total])
un = set([e[1] for e in n])
ur, un, len(total), len(n)

In [ ]:
ddi_valid_comb_en = {
    ('DRUG', 'DRUG')
}

# all four relations are DRUG-DRUG, so only mul mode

In [ ]:
cnt = []

def pp(ddd):
    for k, v in ddd.items():
        cnt.append((k, len(v)))

In [ ]:
CUTOFF=0
dpos, dneg = create_training_samples(ddi_training, None, ddi_valid_comb_en)
pp(dpos)
pp(dneg)
all_in_one(dpos, dneg, dn="2018ddi", do_train=True)
#     partial_unique(dpos, dneg, dn="ddi", do_train=True)
all_in_unique(dpos, dneg, dn="2018ddi", do_train=True)

preds = create_test_samples(ddi_test, None, ddi_valid_comb_en)
pp(preds)
all_in_one(preds, dn="2018ddi", do_train=False)
#     partial_unique(preds, dn="ddi", do_train=False)
all_in_unique(preds, dn="2018ddi", do_train=False)

In [ ]:
c = 0
exs = dpos[0] + dneg[0]
for each in exs:
    if len(each[2].split(" ")) > 200:
        print(each)
        c += 1

In [ ]:
c

In [ ]:
cnt

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
import sys
sys.path.append("../")
sys.path.append("../src")
from src.data_utils import RelationDataFormatSepProcessor

In [ ]:
data_processor = RelationDataFormatSepProcessor(max_seq_len=512)

In [ ]:
data_processor.set_data_dir("../data/2018umass_relation_processed/new_version/2018umass_aiu_th4/cutoff_0/")
data_processor.set_tokenizer(tokenizer)
data_processor.set_num_core(8)

In [ ]:
data = data_processor.get_train_examples()

In [ ]:
len(data)

In [ ]:
"""
single: 24m6s
multi: 12m30s
new_mul: 10m
"""

In [ ]:
from copy import deepcopy

In [ ]:
d1 = deepcopy(data)

In [ ]:
pkl_save(d1, "temp.pkl")

In [ ]:
d1 = pkl_load("temp.pkl")

In [ ]:
len(d1), len(data)

In [ ]:
d2 = []
for each in data:
    d2.extend(each)
len(d2)

In [ ]:
d2 = data

In [ ]:
assert len(d2) == len(d1)

In [ ]:
for s1, s2 in zip(d1, d2):
    assert s1.label == s2.label and s1.text_a == s2.text_a and s1.text_b == s2.text_b, "{}\n{}".format(s1, s2)